In [3]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("fivethirtyeight")
import cvxpy as cvx
from PIL import Image
from copy import deepcopy

In [5]:
R = np.random.randn(125).reshape((25, 5))
print R.shape

(25L, 5L)


In [7]:
M = np.dot(R, R.T)
print M.shape

(25L, 25L)


In [9]:
import random

In [11]:
Mhat60 = deepcopy(M)
for i in xrange(25):
    for j in xrange(25):
        if random.random() < 0.60:
            Mhat60[i, j] = 0

In [13]:
print np.mean(Mhat60 == M)

0.3904


In [26]:
known = np.zeros((25, 25))
for i in xrange(25):
    for j in xrange(25):
        if Mhat60[i, j] != 0:
            known[i, j] = 1
U = cvx.Semidef(25)
obj = cvx.Minimize(cvx.trace(U))
constraints = [cvx.mul_elemwise(known, U) == cvx.mul_elemwise(known, Mhat60), U == U.T]
prob = cvx.Problem(obj, constraints)
#prob.solve(verbose = True, solver=cvx.SCS)

In [27]:
trials = 20
for t in xrange(trials):
    R = np.random.randn(125).reshape((25, 5))
    M = np.dot(R, R.T)
    Mhat60 = deepcopy(M)
    for i in xrange(25):
        for j in xrange(25):
            if random.random() < 0.60:
                Mhat60[i, j] = 0
    U, sigma, V = np.linalg.svd(Mhat60)
    recover = np.matrix(U[:, :5]) * np.diag(sigma[:5]) * np.matrix(V[:5, :])
    print "Trial %d: %0.2f" %(t, np.linalg.norm(recover - M))

Trial 0: 38.86
Trial 1: 46.98
Trial 2: 40.78
Trial 3: 36.41
Trial 4: 38.61
Trial 5: 44.42
Trial 6: 48.24
Trial 7: 38.71
Trial 8: 44.42
Trial 9: 46.13
Trial 10: 35.34
Trial 11: 45.48
Trial 12: 46.93
Trial 13: 43.97
Trial 14: 41.99
Trial 15: 49.59
Trial 16: 39.32
Trial 17: 44.22
Trial 18: 42.41
Trial 19: 49.48
